In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import re
import copy
import shutil

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

# Video Processing

In [7]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [9]:
def split_video(video_path):
    capture = cv2.VideoCapture(video_path)
    fps = copy.deepcopy(capture.get(cv2.CAP_PROP_FPS))
    height = copy.deepcopy(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = copy.deepcopy(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameNr = 0
    while True:
        success, frame = capture.read()
        if success:
            cv2.imwrite(f'fencing_images/frame_{frameNr}.jpg', frame)
        else:
            break
        frameNr += 1
    capture.release()
    
    images = [img for img in os.listdir('fencing_images')] 
    images.sort(key=natural_keys)
    
    return images, fps, height, width

In [11]:
#images, fps, h, w = split_video('fencing.mp4')

In [13]:
def create_video():
    images = [img for img in os.listdir('fencing_images')] 
    images.sort(key=natural_keys)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('fencing_video/complete_video.mp4', fourcc, fps, (int(w), int(h))) 
    for image in images:  
        video.write(cv2.imread('fencing_images/' + image))
    cv2.destroyAllWindows()
    video.release()

In [15]:
#create_video()

In [17]:
def remove_frames(directory_path):
    try:
        for file in os.listdir(directory_path):
            file_path = directory_path + '/' + file
            if os.path.isfile(file_path):
                os.remove(file_path)
        print('all files deleted')
    except:
        print('error')

In [19]:
#remove_frames('fencing_images')

# Adding Model

In [22]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [24]:
%matplotlib inline 
def plot_image(path, index):
    image = cv2.imread(path + images[index])
    results = model(image)
    plt.imshow(np.squeeze(results.render()))
    plt.show()

In [26]:
#plot_image('fencing_images/', 0)

In [28]:
def create_image_folder(images):
    return [cv2.imread('fencing_images/' + img) for img in images]

In [30]:
#all_images = create_image_folder()

In [32]:
#all_images[0].shape

In [34]:
def batch_variables(image_array, batch_size, export_batch_size=False):
    batches = int(len(image_array) / batch_size)
    remainder = len(image_array) % batch_size
    if export_batch_size == False:
        return batches, remainder
    return batch_size, batches, remainder

In [36]:
#batch_size, batches, remainder = batch_variables(all_images, 16, export_batch_size=True)

In [38]:
def model_all_images(model, batch_size, batches, remainder, all_images, h, w):
    batch_images = []
    batch_results = 0
    counter = 0
    while counter < batches*batch_size:
        batch_images = all_images[counter:counter+batch_size]
        batch_results = model(batch_images, size=(int(h),int(w)))
        batch_results.save()
        counter += batch_size
    batch_images = all_images[counter:counter+remainder]
    batch_results = model(batch_images, size=(int(h),int(w)))
    batch_results.save()

In [40]:
#model_all_images(model, batch_size, batches, remainder, all_images, h, w)

In [42]:
def create_names(batch_size):
    res = []
    for i in range(batch_size):
        res.append(f'image{i}.jpg')
    return res

In [44]:
#image_names = create_names(16)

In [46]:
def create_exp(batches):
    res = ['exp']
    for i in range(2, batches+1):
        res.append(f'exp{i}')
    return res

In [48]:
def create_model_video(batch_size, batches, remainder, fps, h, w):
    image_names = create_names(batch_size)
    exp_names = create_exp(batches)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('fencing_video/model_video.mp4', fourcc, fps, (int(w), int(h))) 
    for dir in exp_names[:-1]:
        for image in image_names:
            video.write(cv2.imread(f'runs/detect/{dir}/{image}'))
    if remainder > 0:
        for image in image_names[:remainder]:
            video.write(cv2.imread(f'runs/detect/{exp_names[-1]}/{image}'))
    cv2.destroyAllWindows()
    video.release()
    return image_names, exp_names

In [50]:
#create_model_video(batch_size, batches, remainder, fps, h, w)

In [52]:
def remove_model_images(exp_names):
    try:
        for file in exp_names:
            file_path = f'runs/detect/{file}'
            shutil.rmtree(file_path, ignore_errors=True)
        print('all files deleted')
    except:
        print('error')

In [54]:
#remove_model_images(exp_names)

# Main Function

In [56]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /home/shawn/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-29 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5834MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [60]:
!cd yolov5 && python train.py --img 720 --batch 8 --epochs 500 --data dataset.yml --weights yolov5s.pt --device 0

train: weights=yolov5s.pt, cfg=, data=dataset.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=8, imgsz=720, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 2 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-360-gb40781b3 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5834MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_e

In [58]:
def complete_video_model(video_path, model, batch_size, remove_images=True):
    images, fps, h, w = split_video(video_path)
    all_images = create_image_folder(images)
    batches, remainder = batch_variables(all_images, batch_size)
    model_all_images(model, batch_size, batches, remainder, all_images, h, w)
    image_names, exp_names = create_model_video(batch_size, batches, remainder, fps, h, w)
    if remove_images:
        remove_model_images(exp_names)
        remove_frames('fencing_images')

In [60]:
complete_video_model('fencing.mp4', model, 32, remove_images=False)

/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp2
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp3
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


In [88]:
#remove_model_images(exp_names)
#remove_frames('fencing_images')

# Load Custom Model

In [74]:
custom_model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp10/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/shawn/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-8-29 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5834MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [90]:
complete_video_model('fencing.mp4', custom_model, 32, remove_images=False)

/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp2
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 32 images to runs/detect/exp3
/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


In [78]:
def bounding_box():
    img = cv2.imread('fencing_images/frame_0.jpg')
    res = model(img)
    print(res.xyxy)

In [80]:
bounding_box()

[tensor([[1.91729e+02, 1.45183e+02, 3.73396e+02, 4.30498e+02, 9.19641e-01, 0.00000e+00],
        [9.55854e+02, 1.42111e+02, 1.14825e+03, 4.34414e+02, 9.16369e-01, 0.00000e+00],
        [9.11735e+02, 1.83606e+02, 9.86641e+02, 2.63397e+02, 7.82456e-01, 3.80000e+01],
        [6.05267e+02, 5.63828e+02, 7.49007e+02, 7.14617e+02, 6.36569e-01, 0.00000e+00],
        [3.51442e+02, 2.33691e+02, 3.74154e+02, 2.56046e+02, 2.95318e-01, 3.80000e+01]], device='cuda:0')]


/home/shawn/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
